I got the union to work on the Mac version. I will need to move that over to this one and likewise move the filter method onto the mac one as that was a great find even if I do not remember what it does currently. ah, it may be really useful in fact.

In [ ]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
class Region:
    def __init__(self, *args):
        if len(args) == 3:
            self.definition = [args]
        elif len(args) == 1:
            self.definition = args[0]
        elif len(args) == 0:
            self.definition = []
        else:
            raise ValueError('Pass three values or a list of tuples of three items')
        
    def reduce(self, df: pd.DataFrame):
        return df.loc[self.row_indexer(df)]
    
    def converted(self):
        return self + Region('s_converted', 'eq', 1)
    
    def attached(self):
        return self.converted() + Region('s_attached', 'eq', 1)
    
    def get_elasticity(self, df: pd.DataFrame = df, attach: bool = True, verbose: bool = False, group_by_date: bool = True):
        main_region = self.converted() if attach else self
        dated_summary_data = main_region.reduce(df).groupby(['s_date', 's_insurance_price_vs_original'] if group_by_date else ['s_insurance_price_vs_original'])['s_insurance_ppd', 's_attached' if attach else 's_converted'].mean()
        if attach == True:
            g = (main_region.reduce(df).groupby(['s_date', 's_insurance_price_vs_original'])['s_insurance_ppd','s_converted','s_attached','s_date','s_insurance_price_vs_original'])
            t = g.filter(lambda x: np.sum(x.s_attached) > 1)
            dated_summary_data = t.groupby(['s_date', 's_insurance_price_vs_original'])['s_insurance_ppd','s_attached'].mean()
        if attach == False:
            g = (main_region.reduce(df).groupby(['s_date', 's_insurance_price_vs_original'])['s_insurance_ppd','s_converted','s_attached','s_date','s_insurance_price_vs_original'])
            t = g.filter(lambda x: np.sum(x.s_converted) > 1)
            dated_summary_data = t.groupby(['s_date', 's_insurance_price_vs_original'])['s_insurance_ppd','s_converted'].mean()
            
        dated_summary_data['log_insurance_ppd'] = np.log(dated_summary_data['s_insurance_ppd'])
        if attach:
            dated_summary_data['log_attach_rate'] = np.log(dated_summary_data['s_attached'])
        else:
            dated_summary_data['log_conversion_rate'] = np.log(dated_summary_data['s_converted'])

        model = ols(f"log_{'attach' if attach else 'conversion'}_rate ~ log_insurance_ppd", data=dated_summary_data).fit()
        elasticity = model.params['log_insurance_ppd'], (model.conf_int()[0]['log_insurance_ppd'], model.conf_int()[1]['log_insurance_ppd'])
        
        if verbose:
            print(model.summary())
            print()
            print(f"Modelled elasticity: {elasticity[0]:.3f}, [{elasticity[1][0]:.4f}, {elasticity[1][1]:.4f}]")
            sns.regplot(x='log_insurance_ppd', y='log_attach_rate' if attach else 'log_conversion_rate', data=dated_summary_data)
            plt.show()
                
        return elasticity
                          
    def get_elasticities(self, df: pd.DataFrame = df, verbose: bool = False, group_by_date: bool = True):
        attach = self.get_elasticity(df, attach=True, verbose=verbose, group_by_date=group_by_date)
        conversion = self.get_elasticity(df, attach=False, verbose=verbose, group_by_date=group_by_date)
        return { 'attach': attach, 'conversion': conversion }

    def __add__(self, other):
        return Region(self.definition + other.definition)
#Could this work. if not then how can I do it.                   
    def union(self, other):
        return Region(self.definition).append(Region(other.definition))  
                    
    def __radd__(self, other):
        return Region(other.definition + self.definition)
    
    @classmethod
    def variant(cls, variant):
        return Region('s_variant', 'eq', variant)
                          
    @classmethod
    def experiment(cls, experiment):
        return Region('s_experimentid', 'eq', experiment)
                    
    def row_indexer(self, df: pd.DataFrame):
        condition = pd.Series(True for _ in range(len(df)))
        for (variable, func, arguments) in self.definition:
            if type(arguments) == tuple:
                condition &= getattr(df[variable], func)(*arguments)
            else:
                condition &= getattr(df[variable], func)(arguments)
        return condition
                                      
    def __repr__(self):
        output = ''
        for condition in self.definition:
            a, b, c = condition
            if b == 'eq':
                condition_text = f'{a} == {c}'
            elif b == 'lt':
                condition_text = f'{a} < {c}'
            elif b == 'le':
                condition_text = f'{a} <= {c}'
            elif b == 'gt':
                condition_text = f'{a} > {c}'
            elif b == 'ge':
                condition_text = f'{a} >= {c}'
            elif type(c) is tuple:
                condition_text = f'{a}.{b}{c}'
            else:
                condition_text = f'{a}.{b}({c})'
            output += f'{condition_text} & '
        return output[:-3]
